In [2]:
import pandas as pd
from datamatch import (
    JaroWinklerSimilarity,
    ThresholdMatcher,
    ColumnsIndex,
)

In [3]:
def read_data():
    df = pd.read_csv("../data/OK/aggregate_members_oathkeepers_cleaned.csv")
    return df 

In [4]:
dfa = read_data()

In [5]:
dfa = dfa.fillna("")

In [6]:
dfa.loc[:, "state"] = dfa.state.str.lower().str.strip()
dfa.state.unique()

array(['wy', 'me', 'nm', 'nh', 'ok', 'nc', 'nj', 'ca', 'ga', 'vt', 'ak',
       'id', 'ri', 'tx', 'pa', 'ne', 'ny', 'in', 'mt', 'or', 'wa', 'ma',
       'al', 'va', 'il', 'nv', 'wi', 'fl', 'mi', 'oh', 'co', 'ct', 'sc',
       'la', 'ia', 'ar', 'az', 'tn', 'ut', 'ks', 'ky', 'mn', 'wa.', 'mo',
       'md', 'hi', 'wv', 'ms', 'dc', 'de', 'sd', 'nd', 'ae', '', 'canada',
       'grand island', 'sacramento', 'thailand', 'ne.', 'fl  (az)', 'ma.',
       'la.', 'n.y.', 'pa.', 'iowa', 'bc canada', '??', 'austraila pto',
       'ap', 'lv', 'jacksonville', 'bc', 'oslo', 'alberta, canada',
       'western australia', 'rl', 'carolina beach', '--', 'cn', 'pr',
       'md�', 'florida', '9000', 'stewartstown', 'roosevelt',
       'ca  nonono', 'e7n2v2', 'st croix', 'hamden', 'ontario',
       '607ws/dow, unit: 15173, apo, ap 96205, united states',
       'oh  start', 'mo  end', 'ny11580', '8724', 'ny11766', 'nc28773',
       'sterling heights'], dtype=object)

In [7]:
dfa = dfa[dfa.state.isin(["ct"])]
dfa.loc[:, "notes"] = dfa.notes.str.lower().str.strip()

In [8]:
dfa.loc[:, "name"] = dfa.first_name.fillna("").str.cat(dfa.last_name.fillna(""), sep=" ")

In [9]:
search = dfa[dfa.name.str.contains("k")]
search

,id_number,group_name,bool,city,state,email,undefined_date,notes,first_name,middle_name,last_name,name
416,LT0435,Liberty Tree,No,Old Saybrook,ct,jspeedbump@gmail.com,3/26/2012,,kent,p,johnson,kent johnson
651,LT0659,Liberty Tree,No,Sharon,ct,saac@saac.com,11/12/2012,,richard,j,kopec,richard kopec
1108,LT1122,Liberty Tree,No,Windsor,ct,tjkinder@comcast.net,3/28/2013,,thomas,j,kinderknecht,thomas kinderknecht
1176,LT1198,Liberty Tree,No,Litchfield,ct,tkbowhunter103@gmail.com,4/21/2013,lifetime,troy,david,kaiser,troy kaiser
1333,LT1359,Liberty Tree,No,Ansonia,ct,tattoomark@gmail.com,9/13/2013,,mark,,wood,mark wood
...,...,...,...,...,...,...,...,...,...,...,...,...
36940,Life #574,Life,No,Moosup,ct,dpickett2@yahoo.com,1/24/2013,,david,j,pickett,david pickett
37215,Life #859,Life,No,Enfield,ct,miltystu2004@yahoo.com,10/9/2013,,kevin,,perrone,kevin perrone
37319,Life #964,Life,No,West Hartford,ct,ryanwilk123@gmail.com,12/17/2013,hat/pin,ryan,,wilkinson,ryan wilkinson
37825,Life #1486,Life,No,Windsor Locks,ct,halmersk@gmail.com,42209.97083,hat/pin/coin,keith,d,halmers,keith halmers


In [10]:
search_2 = dfa[dfa.notes.str.contains("officer")]
search_2.notes.unique()

array(["former corrections officer state of connecticut. sirs, i am surprised that this group of men and women have not been included in you list of memberships!! of all people, these are 'frontline' americans ~have no doubt~ consider including1000's of them",
       'many of my patients are police officers, troopers, fbi agents, cia and in the military. i have the ability to spread the word of this organization to my patients. i showed 2 troopers this week the web link.',
       'naval submarine base sub school staff. medically retired 1/18/1980. i was also a volunteer firefighter for 19 years emt for 11 years and an animal control officer for 20 + years.',
       'served as an nbc officer, supply officer and training officer. instructor for command and general staff college for the last ten years of service. police officer in hartford ct for 10 years.',
       'ct state certified fire fighter , ct state haz mat technician, medical response technician, information technology security 

In [11]:
# dfs = [search_1, search_2]
# dfb = pd.concat(dfs, axis=0)
# dfb

In [12]:
def read_misconduct_data():
    df = pd.read_csv("../data/CT/ct_misconduct.csv")
    return df

In [13]:
dfc = read_misconduct_data()

In [14]:

def match_ok_names_to_personnel(ok, agency):
    dfa = ok[["first_name", "last_name"]]
    dfa.loc[:, "first_name"] = dfa.first_name.str.lower().str.strip()
    dfa.loc[:, "last_name"] = dfa.last_name.str.lower().str.strip()
    dfa.loc[:, "fc"] = dfa.first_name.fillna("").map(lambda x: x[:1])
    dfa.loc[:, "lc"] = dfa.last_name.fillna("").map(lambda x: x[:1])

    dfb = agency[["first_name", "last_name"]]
    dfb.loc[:, "first_name"] = dfb.first_name.str.lower().str.strip()
    dfb.loc[:, "last_name"] = dfb.last_name.str.lower().str.strip()
    dfb.loc[:, "fc"] = dfb.first_name.fillna("").map(lambda x: x[:1])
    dfb.loc[:, "lc"] = dfb.last_name.fillna("").map(lambda x: x[:1])

    matcher = ThresholdMatcher(
        ColumnsIndex([]),
        {
            "first_name": JaroWinklerSimilarity(),
            "last_name": JaroWinklerSimilarity(),
        },
        dfa,
        dfb,
    )
    decision = 0.800
    matcher.save_pairs_to_excel(
        ("../data/CT/oathkeeper_names_v_CT_misconduct_data.xlsx"), decision
    )
    return matcher


In [15]:
match = match_ok_names_to_personnel(dfa, dfc)

C:\Users\PC\AppData\Local\Temp\ipykernel_14624\2697142198.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfa.loc[:, "first_name"] = dfa.first_name.str.lower().str.strip()
C:\Users\PC\AppData\Local\Temp\ipykernel_14624\2697142198.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfa.loc[:, "last_name"] = dfa.last_name.str.lower().str.strip()
C:\Users\PC\AppData\Local\Temp\ipykernel_14624\2697142198.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try us

In [ ]:
# def read_matches():
#     df = pd.read_excel("../data/HI/oathkeeper_names_v_HI_misconduct_data.xlsx")
#     return df 

In [ ]:
# matches = read_matches()

In [ ]:
# matches